In [1]:
"""CIFAR example using input pipelines."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Standard Imports
import tensorflow as tf


def model_fn(features, labels, mode, params):
    """A simple CNN."""

    conv1 = tf.layers.conv2d(
      inputs=features,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(
      inputs=conv1,
      pool_size=[2, 2],
      strides=2,
      padding="same")

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    pool2 = tf.layers.max_pooling2d(
      inputs=conv2,
      pool_size=[2, 2],
      strides=2,
      padding="same")

    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])

    dense = tf.layers.dense(
      inputs=pool2_flat,
      units=384,
      activation=tf.nn.relu)

    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=10)

    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    with tf.name_scope('loss'): 
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    tf.summary.scalar('loss', loss)
    
    predictions = tf.argmax(logits, 1)
    correct_prediction = tf.equal(tf.cast(labels, tf.int32), tf.cast(predictions, tf.int32))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
#     eval_metric_ops = {
#                      "accuracy": tf.metrics.accuracy(labels, predictions)
#     }
    #     learning_rate = tf.train.exponential_decay(
    #       FLAGS.learning_rate, tf.train.get_global_step(), 25000, 0.96)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=params['learning_rate'])

    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    tf.summary.merge_all()
    return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
    )


def input_fn(params):
    """A simple input_fn using the experimental input pipeline."""

    batch_size = params["batch_size"]
    def _parse_function(example_proto):
        features={
          'image/encoded': tf.FixedLenFeature((), tf.string, default_value=""),
          'image/format': tf.FixedLenFeature((), tf.string, default_value=""),
          'image/class/label': tf.FixedLenFeature((), tf.int64, default_value=0),
          'image/height': tf.FixedLenFeature((), tf.int64, default_value=0),
          'image/width': tf.FixedLenFeature((), tf.int64, default_value=0),
        }
        parsed_features = tf.parse_single_example(example_proto, features)
        image = tf.image.decode_png(parsed_features["image/encoded"], dtype=tf.uint8, channels=3)
#         image.set_shape([3*32*32])
        # Normalize the values of the image from the range [0, 255] to [-0.5, 0.5]
        image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
#         image = tf.transpose(tf.reshape(image, [3, 32*32]))
        label = tf.cast(parsed_features["image/class/label"], tf.int32)
        return image, label

    dataset = tf.contrib.data.TFRecordDataset(["../data/cifar10_data/cifar10_train.tfrecord"])
    dataset = dataset.repeat().map(_parse_function).batch(batch_size)
    images, labels = dataset.make_one_shot_iterator().get_next()
 
    return (
      images,
      tf.reshape(labels, [batch_size])
      ) 

In [2]:
def main(unused_argv):
    
    del unused_argv  # Unused
    
    run_config = tf.contrib.learn.RunConfig(save_summary_steps=100, model_dir='../tmp/tf')
    tf.logging.set_verbosity(tf.logging.INFO)
#     with  tf.Session("") as  sess:
    estimator = tf.estimator.Estimator(model_fn=model_fn, 
                                       params={'learning_rate':0.05, 'batch_size':64}, config=run_config)
    estimator.train(input_fn=input_fn, max_steps=20000)

In [3]:
# tf.flags.DEFINE_integer("save_checkpoints_secs", None,
#                         "Seconds between checkpoint saves")
# tf.flags.DEFINE_string("model_dir", '../model_dir/', "Estimator model_dir")
FLAGS = tf.flags.FLAGS

In [4]:
main('empty')
# tf.app.run()

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x119ed9d68>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '../tmp/tf'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ../tmp/tf/model.ckpt.
INFO:tensorflow:loss = 2.30111, step = 1
INFO:tensorflow:global_step/sec: 2.55412
INFO:tensorflow:loss = 2.05459, step = 101 (39.155 sec)
INFO:tensorflow:global_step/sec: 2.56006
INFO:tensorflow:loss = 1.66352, step = 201 (39.061 sec)
INFO:tensorflow:global